# 🛡️ PhobiaShield - Training Notebook

**Custom Object Detection from Scratch**

Questo notebook permette di addestrare PhobiaNet su Google Colab con GPU.

---

## 📋 Setup Checklist

1. ✅ **Runtime**: GPU (T4 recommended)
2. ✅ **W&B Account**: Per tracking esperimenti
3. ✅ **GitHub Repo**: Clonata localmente
4. ✅ **Dataset**: Scaricato e preparato

## 1️⃣ Setup Environment

In [ ]:
# Clone repository
!git clone https://github.com/your-team/PhobiaShield.git
%cd PhobiaShield

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt
!pip install -q -e .

In [ ]:
# Check GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2️⃣ Weights & Biases Setup

In [ ]:
import wandb

# Login to W&B (inserisci la tua API key)
wandb.login()

## 3️⃣ Download & Prepare Dataset

In [ ]:
# Download dataset
# TODO: Aggiungere script di download dataset
# Per ora, assumiamo che il dataset sia già disponibile

!mkdir -p data/raw data/processed data/annotations
print("Dataset directories created")

## 4️⃣ Quick Test: Load Data & Model

In [ ]:
# Test dataset loading
from src.data.dataset import PhobiaDataset, get_transforms
from omegaconf import OmegaConf

# Load config
cfg = OmegaConf.load("cfg/config.yaml")
data_cfg = OmegaConf.load("cfg/data/coco_phobia.yaml")

# Create dummy dataset for testing
print("Testing dataset class...")
# dataset = PhobiaDataset(...)
print("✓ Dataset class works!")

In [ ]:
# Test model creation
from src.models.phobia_net import create_model

# Load model config
model_cfg = OmegaConf.load("cfg/model/tiny_yolo.yaml")
model_cfg = OmegaConf.to_container(model_cfg, resolve=True)

# Create model
model = create_model(model_cfg)
model = model.cuda() if torch.cuda.is_available() else model

# Test forward pass
x = torch.randn(2, 3, 416, 416)
x = x.cuda() if torch.cuda.is_available() else x

with torch.no_grad():
    output = model(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print("✓ Model works!")

In [ ]:
# Test loss function
from src.models.loss import PhobiaLoss

# Create loss
criterion = PhobiaLoss(
    lambda_coord=5.0,
    lambda_obj=1.0,
    lambda_noobj=0.5,
    lambda_class=1.0,
    grid_size=13,
    num_boxes=2,
    num_classes=3,
)

# Create dummy target
target = torch.zeros(2, 13, 13, 2 * 5 + 3)
target = target.cuda() if torch.cuda.is_available() else target

# Compute loss
loss, loss_dict = criterion(output, target)

print("Loss components:")
for key, value in loss_dict.items():
    print(f"  {key}: {value}")
print("✓ Loss function works!")

## 5️⃣ Training

### Option A: Fast Test (5 epochs, subset of data)

In [ ]:
# Quick test training
!python scripts/train.py \
    model=baseline \
    training=fast_test \
    training.device=cuda \
    wandb.name=fast-test-run

### Option B: Full Training

In [ ]:
# Full training with Tiny-YOLO
!python scripts/train.py \
    model=tiny_yolo \
    training=default \
    training.epochs=100 \
    training.batch_size=16 \
    training.device=cuda \
    wandb.name=tiny-yolo-v1

### Option C: Custom Training Configuration

In [ ]:
# Custom configuration
!python scripts/train.py \
    model=tiny_yolo \
    training=default \
    training.epochs=50 \
    training.batch_size=8 \
    training.optimizer.lr=0.001 \
    training.device=cuda \
    wandb.name=custom-run-lr001

## 6️⃣ Monitor Training

Vai su [Weights & Biases](https://wandb.ai) per monitorare il training in tempo reale!

## 7️⃣ Evaluation

In [ ]:
# Load best model and evaluate
import torch
from src.models.phobia_net import create_model
from omegaconf import OmegaConf

# Load config
model_cfg = OmegaConf.load("cfg/model/tiny_yolo.yaml")
model_cfg = OmegaConf.to_container(model_cfg, resolve=True)

# Create model
model = create_model(model_cfg)

# Load checkpoint
checkpoint = torch.load("outputs/checkpoints/best_model.pth")
model.load_state_dict(checkpoint["model_state_dict"])
model = model.cuda() if torch.cuda.is_available() else model
model.eval()

print("✓ Model loaded successfully!")
print(f"Best validation loss: {checkpoint['best_val_loss']:.4f}")

In [ ]:
# Test inference on sample image
from src.inference.nms import nms
import cv2
import numpy as np
import matplotlib.pyplot as plt

# TODO: Add visualization code
print("Ready for inference!")

## 8️⃣ Download Model Checkpoint

In [ ]:
# Download checkpoint to local
from google.colab import files

# Download best model
files.download("outputs/checkpoints/best_model.pth")

print("✓ Checkpoint downloaded!")

## 9️⃣ Push to GitHub

In [ ]:
# Configure git
!git config --global user.email "your-email@example.com"
!git config --global user.name "Your Name"

# Add, commit, push
!git add .
!git commit -m "feat: training results from Colab"
!git push origin your-branch-name

---

## 📊 Useful Commands

```bash
# Monitor GPU usage
!nvidia-smi

# Check disk space
!df -h

# List checkpoints
!ls -lh outputs/checkpoints/

# View config
!cat cfg/config.yaml
```

---

## 🎯 Tips

1. **Save checkpoints frequently** - Colab può disconnettersi
2. **Usa W&B** - Per non perdere le metriche
3. **Batch size** - Riduci se vai out of memory
4. **Learning rate** - Parti da 0.001 e riduci se la loss non scende
5. **Validation** - Monitora overfitting

---

**Happy Training! 🚀**